In [2]:
import cv2
import numpy as np
import os

In [3]:
#Set Width and Height of image
image_x, image_y = 50, 50

In [4]:
#Function to create folder for each gesture
def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)

In [5]:
#Start webcam
cap = cv2.VideoCapture(0)
fbag = cv2.createBackgroundSubtractorMOG2()

In [6]:
#Function to store gestures        
def main(g_id):
    #Max pics of gesture allowed
    total_pics = 500
    cap = cv2.VideoCapture(0)
    #Particular area of capturing gesture
    x, y, w, h = 300, 50, 350, 350

    create_folder("Gestures/" + str(g_id))
    pic_no = 0
    flag_start_capturing = False
    frames = 0

    while True:
        #Read frame from webcam
        ret, frame = cap.read()
        #Flip the read frame to get correct orientation
        frame = cv2.flip(frame, 1)
        #Convert to HSV Format
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        #THRESH SCREEN
        #Set mask in range of skin color
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(frame, frame, mask=mask2)
        #Convert to grey scale
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        #Initialize kernel
        kernel_square = np.ones((5, 5), np.uint8)
        #Improve the quality of captured gesture
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening=cv2.morphologyEx(dilation,cv2.MORPH_CLOSE,kernel_square)

        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)
        #Set thresh according to particular area
        thresh = thresh[y:y + h, x:x + w]
        #Find cotours in thresh
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]

        #IF contours exixts
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            #Condition for starting the process of capturing gesture
            if cv2.contourArea(contour) > 10000 and frames > 50:
                x1, y1, w1, h1 = cv2.boundingRect(contour)
                pic_no += 1
                save_img = thresh[y1:y1 + h1, x1:x1 + w1]
                if w1 > h1:
                    save_img = cv2.copyMakeBorder(save_img, int((w1 - h1) / 2), int((w1 - h1) / 2), 0, 0,
                                                  cv2.BORDER_CONSTANT, (0, 0, 0))
                elif h1 > w1:
                    save_img = cv2.copyMakeBorder(save_img, 0, 0, int((h1 - w1) / 2), int((h1 - w1) / 2),
                                                  cv2.BORDER_CONSTANT, (0, 0, 0))
                save_img = cv2.resize(save_img, (image_x, image_y))
                cv2.putText(frame, "Capturing...", (30, 60), cv2.FONT_HERSHEY_TRIPLEX, 2, (127, 255, 255))
                cv2.imwrite("Gestures/" + str(g_id) + "/" + str(pic_no) + ".jpg", save_img)

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, str(pic_no), (30, 400), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (127, 127, 255))
        #FRAME SCREEN Heading
        cv2.imshow("Capturing gesture", frame)
        #THRESH SCREEN Heading
        cv2.imshow("thresh", thresh)
        #Start the process when 'c' is pressed
        keypress = cv2.waitKey(1)
        if keypress == ord('c'):
            if flag_start_capturing == False:
                flag_start_capturing = True
            else:
                flag_start_capturing = False
                frames = 0
        if flag_start_capturing == True:
            frames += 1
        if pic_no == total_pics:
            break

In [7]:
g_id = input("Enter gesture number: ")
main(g_id)

Enter gesture number: 20
